In [78]:
cd learning/Resume Analyser/4.version/

C:\Users\Lenovo\learning\Resume Analyser\4.version


In [3]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 7010-C44B

 Directory of C:\Users\Lenovo\learning\Resume Analyser\4.version

11/20/2024  01:31 PM    <DIR>          .
11/04/2024  08:36 PM    <DIR>          ..
11/20/2024  06:34 AM    <DIR>          .ipynb_checkpoints
11/20/2024  06:33 AM    <DIR>          __pycache__
11/19/2024  02:12 PM             2,031 constants.py
11/20/2024  01:12 PM            76,468 funct_test.ipynb
11/11/2024  07:34 PM            23,719 HR_analytics.ipynb
11/11/2024  07:34 PM         5,213,294 job_final.csv
11/04/2024  08:38 PM    <DIR>          marker
11/20/2024  01:31 PM               411 prerequisite.py
11/19/2024  02:32 PM             3,673 pyresparser.py
11/20/2024  01:30 PM               103 requirements.txt
11/20/2024  12:39 PM                68 skills.csv
11/14/2024  10:13 PM    <DIR>          trainning
11/19/2024  02:29 PM            16,227 utils.py
               9 File(s)      5,335,994 bytes
               6 Dir(s)  193,583,902,720 bytes fre

In [7]:
!pip install -r requirements.txt

  Using cached nltk-3.2.4.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pdfminer.six-20240706-py3-none-any.whl.metadata (4.1 kB)
  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached cryptography-43.0.3-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
Using cached pdfminer.six-202

In [11]:
%run prerequisite.py

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_dat

In [17]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 8.5 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.8 MB 9.9 MB/s eta 0:00:02
     ---------------------------- ----------- 9.2/12.8 MB 18.4 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 20.6 MB/s eta 0:00:00


In [18]:
import constants as cs

from spacy.matcher import Matcher
import spacy

import re
from collections import defaultdict
from urllib.parse import urlparse

import pandas as pd
import os

## Extract name

In [19]:
pattern = cs.NAME_PATTERN
pattern

[{'POS': 'PROPN'}]

In [20]:
def extract_name(doc, matcher):
    '''
    Helper function to extract name from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param matcher: object of `spacy.matcher.Matcher`
    :return: string of full name
    '''
    # Add a pattern to match proper nouns (PROPN)
    pattern = cs.NAME_PATTERN
    matcher.add('NAME', [pattern])

    # Apply the matcher to the doc
    matches = matcher(doc)

    # List to store full names
    full_names = []

    # Temporary variable to track the end position of the last matched name
    last_end_index = -1

    # Iterate over the matches
    for match_id, start, end in matches:
        # If the current match is contiguous with the last match, append the name
        if start == last_end_index:
            full_names[-1] += " " + doc[start:end].text  # Merge the names
        else:
            full_names.append(doc[start:end].text)  # Add a new name

        # Update the last_end_index to the end of the current match
        last_end_index = end

    return full_names


text = "John Doe is a software developer. And Scatter William Smith is an computer engineering."
nlp = spacy.load('en_core_web_sm')
matchers = Matcher(nlp.vocab)
full_names = extract_name(nlp(text), matcher=matchers)
print(full_names)

['John Doe', 'Scatter William Smith']


In [21]:
text = "John Doe is a software developer. And Scatter Will Smith is an computer engineering."
matchers = Matcher(nlp.vocab)
matchers.add('NAME', [[{'POS': 'PROPN'}]])
matchers(nlp(text))

[(424143666773229379, 0, 1),
 (424143666773229379, 1, 2),
 (424143666773229379, 8, 9),
 (424143666773229379, 10, 11)]

## Extract link

In [22]:
# Example text containing domain names with paths and some with protocols
text = """
Here are some domain names with and without protocols:
http://example.com
https://mywebsite.org/about
subdomain.example.net/products?category=books
another-site.co.uk/contact?ref=home
ftp://fileserver.com/path/to/file
"""

# Regular expression to match domain names with optional paths or query parameters
domain_with_path_pattern = r'\b(?:[A-Za-z][A-Za-z0-9+.-]*://)?[A-Za-z0-9.-]+\.[a-zA-Z]{2,}(?:/[A-Za-z0-9&%_\-.?=]*)*\b'

# Extracting all domain names with or without protocols
domains_with_paths = re.findall(domain_with_path_pattern, text)

# Prepare the final list with https:// if no protocol exists
final_links = []
for link in domains_with_paths:
    # If the link doesn't have a protocol (i.e., it doesn't start with 'http://', 'https://', etc.)
    if not re.match(r'^[a-zA-Z][a-zA-Z0-9+.-]*://', link):  # No protocol
        link = 'https://' + link  # Add https:// as the default protocol
    final_links.append(link)

# Print the final list of links
for link in final_links:
    print(link)


http://example.com
https://mywebsite.org/about
https://subdomain.example.net/products?category=books
https://another-site.co.uk/contact?ref=home
ftp://fileserver.com/path/to/file


In [23]:
def extract_links(text):
    # Regular expression to match domain names with optional protocols and paths
    domain_with_path_pattern = cs.LINK_PATTERN
    # Extract all the links (with or without protocol)
    links = re.findall(domain_with_path_pattern, text)

    # Dictionary to hold domain as key and list of links as value
    domain_dict = defaultdict(list)

    # Iterate through each link
    for link in links:
        # If the link doesn't have a protocol, add https:// by default
        if not re.match(r'^[a-zA-Z][a-zA-Z0-9+.-]*://', link):  # No protocol
            link = 'https://' + link

        # Parse the link to get the domain name (without protocol)
        parsed_url = urlparse(link)
        domain_name = parsed_url.netloc

        # Append the link to the list of the corresponding domain
        domain_dict[domain_name].append(link)

    return domain_dict

# Example text containing domain names with paths and some with protocols
text = """
Here are some domain names with and without protocols:
http://example.com
https://mywebsite.org/about
subdomain.example.net/products?category=books
another-site.co.uk/contact?ref=home
ftp://fileserver.com/path/to/file
"""

# Call the function and get the domain dictionary
domain_links = extract_links(text)

# Print the resulting dictionary
for domain, links in domain_links.items():
    print(f"{domain}:")
    for link in links:
        print(f"  {link}")


example.com:
  http://example.com
mywebsite.org:
  https://mywebsite.org/about
subdomain.example.net:
  https://subdomain.example.net/products?category=books
another-site.co.uk:
  https://another-site.co.uk/contact?ref=home
fileserver.com:
  ftp://fileserver.com/path/to/file


## Extract phone number

In [24]:
def extract_phone_numbers(text):
    # Regular expression to match phone numbers (supports various formats)
    phone_pattern = r'(\+?\d{1,3}[-.\s]?)?(\(?\d{1,4}\)?[-.\s]?)?(\d{1,4})[-.\s]?(\d{1,4})[-.\s]?(\d{1,4})'

    # Find all matches in the text
    phone_numbers = re.findall(phone_pattern, text)

    # Clean up the result to return full phone numbers (combining the matched parts)
    full_phone_numbers = [''.join(number) for number in phone_numbers]

    return full_phone_numbers

# Example text with phone numbers
text = """
You can reach me at +1-800-555-5555 or (800) 555-5555.
My office number is 555 555 5555. Call me at +44 7911 123456.
Another contact: 123.456.7890. Or just use 800-123-4567.
"""

# Extract phone numbers from the text
phones = extract_phone_numbers(text)

# Print the extracted phone numbers
for phone in phones:
    print(phone)


+1-800-5555555
(800) 5555555
555 555 5555
+44 7911 123456
123.456.7890
800-123-4567


## Extract education

In [25]:
# Simulated constants
class cs:
    EDUCATION = {"BACHELOR", "MASTERS", "PHD", "MBA", "BSC", "MSC"}
    STOPWORDS = {"AND", "OF", "THE"}
    YEAR = r'\b(19|20)\d{2}\b'  # Regex pattern to match years like 1999, 2020

# Function to test
def extract_education(nlp_text):
    '''
    Helper function to extract education from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :return: tuple of education degree and year if year is found
             else only returns education degree
    '''
    edu = {}
    # Extract education degree
    try:
        for index, token in enumerate(nlp_text):
            text_cleaned = re.sub(r'[?|$|.|!|,]', r'', token.text)
            if text_cleaned.upper() in cs.EDUCATION and text_cleaned.upper() not in cs.STOPWORDS:
                # Collect surrounding context (current and next 5 tokens for simplicity)
                context = " ".join([t.text for t in nlp_text[index:index + 6]])
                edu[text_cleaned.upper()] = context
    except IndexError:
        pass

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(cs.YEAR), edu[key])
        if year:
            education.append((key, year.group(0)))  # Include degree and year
        else:
            education.append(key)  # Include degree only
    return education

# Test with spacy
nlp = spacy.load("en_core_web_sm")
test_text = "John completed his Bachelor of Science in 2015 and his Masters in 2020."
nlp_text = nlp(test_text)

# Execute the function
result = extract_education(nlp_text)
print(result)


[('BACHELOR', '2015'), ('MASTERS', '2020')]


## Extract skill

In [26]:
# Mock skills data
skills_data = {"Skill": ["python", "data analysis", "machine learning", "nlp", "deep learning"]}
skills_file_path = "skills.csv"

# Save the mock skills data to a CSV file
pd.DataFrame(skills_data).to_csv(skills_file_path, index=False)

# Function to extract skills
def extract_skills(nlp_text, noun_chunks, skills_file=None):
    '''
    Helper function to extract skills from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :param skills_file: path to skills file (optional)
    :return: list of skills extracted
    '''
    tokens = [token.text for token in nlp_text if not token.is_stop]
    if not skills_file:
        raise ValueError("Skills file is required.")
    else:
        data = pd.read_csv(skills_file)
    skills = list(data['Skill'].str.lower())  # Convert skills to lowercase for matching
    skillset = []
    # Check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    # Check for bi-grams and tri-grams
    for chunk in noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in skills:
            skillset.append(chunk_text)
    return [i.capitalize() for i in set(skillset)]

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Test text
test_text = "I have experience in Python, data analysis, and machine learning."
nlp_text = nlp(test_text)
noun_chunks = list(nlp_text.noun_chunks)

# Test the function
result = extract_skills(nlp_text, noun_chunks, skills_file_path)
print("Extracted Skills:", result)


Extracted Skills: ['Python', 'Data analysis', 'Machine learning', 'Python']


## Extract profession

In [97]:
# Example content for cs.RESUME_SECTIONS_PROFESSIONAL
class cs:
    RESUME_SECTIONS_PROFESSIONAL = {"experience", "education", "skills", "certifications", "projects"}

# Function to extract entity sections from the resume
def extract_entity_sections_professional(text):
    '''
    Helper function to extract all the raw text from sections of
    resume specifically for professionals

    :param text: Raw text of resume
    :return: dictionary of entities
    '''
    text_split = [i.strip() for i in text.split('\n')]
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(cs.RESUME_SECTIONS_PROFESSIONAL)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in cs.RESUME_SECTIONS_PROFESSIONAL:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
    return entities

# Testing the function
entities = extract_entity_sections_professional(__text_raw)
for k, v in entities.items():
    print(k.upper(), ':')
    print('\n    -'.join(v))


EXPERIENCE :
writing andoptimizing SQL code and Stored Procedures, creating functions, views,
    -triggers and indexes.
    -Cloud  platform:  Worked  on  Microsoft  Azure  cloud  services  like  Document  DB,  SQL
    -Azure, StreamAnalytics, Event hub,  Power BI, Web Job, Web App, Power BI, Azure
    -data lake analytics(U-SQL).
    -Big Data: Worked Azure data lake store/analytics for big data processing and Azure
    -data factoryto schedule U-SQL jobs. Designed and developed end to end  big data
    -solution for data insights.
    -Willing to relocate: Anywhere
    -WORK EXPERIENCESoftware Engineer
    -Microsoft - Manchester, UK.
    -December 2015 to Present
    -1. Microsoft Rewards Live dashboards:
    -Description:  -  Microsoft  rewards  is  loyalty  program  that  rewards  Users  for
    -browsing  and  shopping  online.  Microsoft  Rewards  members  can  earn  points  when
    -searching  with  Bing,  browsing  with  Microsoft  Edge  and  making  purchases  at  the
    -

## Extract experience

In [112]:
import nltk
nltk.data.path.append('C:/Users/Lenovo/nltk_data')  # Replace with your path
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [113]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import constants as cs

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [114]:
def extract_experience(resume_text):
    '''
    Helper function to extract experience from resume text

    :param resume_text: Plain resume text
    :return: list of experience
    '''
    # Initialize lemmatizer and stopwords
    wordnet_lemmatizer = WordNetLemmatizer()
    stop_words = cs.STOPWORDS

    # Word tokenization
    word_tokens = word_tokenize(resume_text)

    # Remove stop words and lemmatize
    filtered_sentence = [w for w in word_tokens if w not in stop_words and wordnet_lemmatizer.lemmatize(w) not in stop_words]

    # POS tagging
    sent = nltk.pos_tag(filtered_sentence)

    # Define regex pattern to capture proper nouns (NNP)
    cp = nltk.RegexpParser('P: {<NNP>+}')
    _cs = cp.parse(sent)

    # Extract experience-related chunks (proper nouns)
    test = []
    for vp in list(_cs.subtrees(filter=lambda x: x.label() == 'P')):
        test.append(" ".join([i[0] for i in vp.leaves() if len(vp.leaves()) >= 2]))

    # Search the word 'experience' in the chunk and return text after it
    experience = [x[x.lower().index('experience') + 10:] for i, x in enumerate(test) if x and 'experience' in x.lower()]
    
    return experience

# Example Resume Text for Testing
resume_text = """
John Doe
Software Engineer at ABC Corp.
Responsible for designing and developing web applications.

Experience:
- Worked at XYZ Ltd. as a Senior Developer for 5 years.
- Led the development of a major e-commerce platform.
- Managed a team of 5 developers.
"""
# Call the function to extract experience
experience = extract_experience(resume_text)

# Output the result
for exp in experience:
    print(exp)

LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - 'C:\\Users\\Lenovo/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Lenovo\\anaconda3\\envs\\resumeparser\\nltk_data'
    - 'C:\\Users\\Lenovo\\anaconda3\\envs\\resumeparser\\lib\\nltk_data'
    - 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data'
    - 'C:/Users/Lenovo/nltk_data'
**********************************************************************

In [115]:
import nltk
print(nltk.data.path)

['C:\\Users\\Lenovo/nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data', 'C:\\Users\\Lenovo\\anaconda3\\envs\\resumeparser\\nltk_data', 'C:\\Users\\Lenovo\\anaconda3\\envs\\resumeparser\\lib\\nltk_data', 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data', 'C:/Users/Lenovo/nltk_data']


In [ ]:
def extract_experience(resume_text):
    '''
    Helper function to extract experience from resume text

    :param resume_text: Plain resume text
    :return: list of experience
    '''
    wordnet_lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # word tokenization
    word_tokens = nltk.word_tokenize(resume_text)

    # remove stop words and lemmatize
    filtered_sentence = [
            w for w in word_tokens if w not
            in stop_words and wordnet_lemmatizer.lemmatize(w)
            not in stop_words
        ]
    sent = nltk.pos_tag(filtered_sentence)

    # parse regex
    cp = nltk.RegexpParser('P: {<NNP>+}')
    cs = cp.parse(sent)

    # for i in cs.subtrees(filter=lambda x: x.label() == 'P'):
    #     print(i)

    test = []

    for vp in list(
        cs.subtrees(filter=lambda x: x.label() == 'P')
    ):
        test.append(" ".join([
            i[0] for i in vp.leaves()
            if len(vp.leaves()) >= 2])
        )

    # Search the word 'experience' in the chunk and
    # then print out the text after it
    x = [
        x[x.lower().index('experience') + 10:]
        for i, x in enumerate(test)
        if x and 'experience' in x.lower()
    ]
    return x


## Use model pre-train

In [27]:
pwd

'C:\\Users\\Lenovo\\learning\\Resume Analyser\\4.version'

In [41]:
!pip install spacy_transformers==1.3.5

In [42]:
!pip install transformers==4.36.2

In [49]:
!pip show torch

Name: torch
Version: 2.5.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: c:\users\lenovo\anaconda3\envs\resumeparser\lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: spacy-transformers


In [53]:
!pip install -U spacy-transformers

In [59]:
# (which is NVIDIA-specific)
path = 'trainning/_model/hyper_model/model-best'
model = spacy.load(path)

AttributeError: 'NoneType' object has no attribute 'full'

In [63]:
# Test example
main_str = '''Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response
Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response'''


In [68]:
for doc in model.pipe(main_str, disable=["tagger", "parser"]):
  for ent in doc.ents:
    print([(ent.text, ent.label_) for ent in doc.ents])

[('7', 'CARDINAL')]
[('8', 'CARDINAL')]
[('6', 'CARDINAL')]
[('9', 'CARDINAL')]
[('2', 'CARDINAL')]
[('1', 'CARDINAL')]
[('2', 'CARDINAL')]
[('3', 'CARDINAL')]
[('2', 'CARDINAL')]
[('0', 'CARDINAL')]
[('1', 'CARDINAL')]
[('7', 'CARDINAL')]
[('F', 'ORG')]
[('9', 'CARDINAL')]
[('5', 'CARDINAL')]
[('0', 'CARDINAL')]
[('0', 'CARDINAL')]
[('0', 'CARDINAL')]
[('2', 'CARDINAL')]
[('0', 'CARDINAL')]
[('1', 'CARDINAL')]
[('4', 'CARDINAL')]
[('2', 'CARDINAL')]
[('0', 'CARDINAL')]
[('0', 'CARDINAL')]
[('9', 'CARDINAL')]
[('2', 'CARDINAL')]
[('0', 'CARDINAL')]
[('1', 'CARDINAL')]
[('4', 'CARDINAL')]
[('7', 'CARDINAL')]
[('0', 'CARDINAL')]
[('F', 'ORG')]
[('F', 'ORG')]
[('2', 'CARDINAL')]
[('0', 'CARDINAL')]
[('0', 'CARDINAL')]
[('7', 'CARDINAL')]
[('2', 'CARDINAL')]
[('0', 'CARDINAL')]
[('0', 'CARDINAL')]
[('9', 'CARDINAL')]
[('F', 'ORG')]
[('F', 'ORG')]
[('F', 'ORG')]
[('F', 'ORG')]
[('F', 'ORG')]
[('F', 'ORG')]
[('7', 'CARDINAL')]
[('8', 'CARDINAL')]
[('6', 'CARDINAL')]
[('9', 'CARDINAL')]
[('2'

In [69]:
main_str

'Manufacturing Production Manager Resume\nDesired Industry Manufacturing\nSpiderID 78692\nDesired Job Location Windsor Colorado\nDate Posted 1 23 2017\nType of Position Full Time Permanent\nAvailability Date Immediately\nDesired Wage 95000\nU S Work Authorization Yes\nJob Level Management Manager Director\nWilling to Travel\nHighest Degree Attained Other\nWilling to Relocate Yes\nObjective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams 

In [70]:
def extract_entity_sections_grad_(text):
    # Load the spaCy model
    nlp = spacy.load('en_core_web_sm')
    
    # Process the input text
    doc = nlp(text)

    # Initialize an empty dictionary to store entities
    entities = {}

    # Iterate over the entities found in the doc
    for ent in doc.ents:
        # If the entity label is not already in the dictionary, add it with the entity text as a list
        if ent.label_ not in entities:
            entities[ent.label_] = [ent.text]
        else:
            # Otherwise, append the entity text to the existing list
            entities[ent.label_].append(ent.text)
    
    return entities

print(extract_entity_sections_grad_(main_str))

{'PERSON': ['Resume\nDesired Industry Manufacturing', 'Served', 'Functioned', 'Fort Collis', 'Mar', 'Clerk Night Crew', 'Resume\nDesired Industry Manufacturing', 'Served', 'Functioned', 'Fort Collis', 'Mar', 'Clerk Night Crew'], 'DATE': ['78692', '2017', 'Jan 2014', '2009', '2007', '78692', '2017', 'Jan 2014', '2009', '2007'], 'GPE': ['Colorado', 'Proficient', 'Colorado', 'Proficient'], 'ORG': ['U S Work Authorization', 'Relocate', 'Adept', 'Microsoft Office', 'Project Management', 'Microsoft Visio', 'Initiated', 'Shells Lighthouse', 'Anheuser Busch', 'COBrew House', 'Siemens', 'Strictly', 'ExperienceUnited States Airways', 'Denver CoCustomer Service', 'Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery', 'Range Community College', 'Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites', 'Rock Springs', 'Skills Manufacturing Production', 'Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturin

## Extract text

In [71]:
import utils

In [79]:
pwd

'C:\\Users\\Lenovo\\learning\\Resume Analyser\\4.version'

In [82]:
__text_raw = utils.extract_text('../data/data_summarise_info/test/Smith Resume.pdf')
__text_raw

' Michael Smith \nBI / Big Data/ Azure \nManchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f \n\n10+  years  of  Experience  in  Designing,  Development,  Administration,  Analysis, \nManagement  inthe  Business  Intelligence  Data  warehousing,  Client  Server \nTechnologies, Web-based Applications, cloud solutions and Databases. \nData warehouse: Data analysis, star/ snow flake schema data modeling and design \nspecific todata warehousing and business intelligence environment. \nDatabase:  Experience  in  database  designing,  scalability,  back-up  and  recovery, \nwriting andoptimizing SQL code and Stored Procedures, creating functions, views, \ntriggers and indexes.  \nCloud  platform:  Worked  on  Microsoft  Azure  cloud  services  like  Document  DB,  SQL \nAzure, StreamAnalytics, Event hub,  Power BI, Web Job, Web App, Power BI, Azure \ndata lake analytics(U-SQL). \nBig Data: Worked Azure data lake store/analytics for big data processing and Azure \ndata 

In [83]:
__text = ' '.join(__text_raw.split())
__text

'Michael Smith BI / Big Data/ Azure Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f 10+ years of Experience in Designing, Development, Administration, Analysis, Management inthe Business Intelligence Data warehousing, Client Server Technologies, Web-based Applications, cloud solutions and Databases. Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific todata warehousing and business intelligence environment. Database: Experience in database designing, scalability, back-up and recovery, writing andoptimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL). Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factoryto schedule U-SQL jobs. Designed and developed end to end big 

In [84]:
utils.get_number_of_pages('../data/data_summarise_info/test/Smith Resume.pdf')

3

EXPERIENCE :
writing andoptimizing SQL code and Stored Procedures, creating functions, views,
    -triggers and indexes.
    -Cloud  platform:  Worked  on  Microsoft  Azure  cloud  services  like  Document  DB,  SQL
    -Azure, StreamAnalytics, Event hub,  Power BI, Web Job, Web App, Power BI, Azure
    -data lake analytics(U-SQL).
    -Big Data: Worked Azure data lake store/analytics for big data processing and Azure
    -data factoryto schedule U-SQL jobs. Designed and developed end to end  big data
    -solution for data insights.
    -Willing to relocate: Anywhere
    -WORK EXPERIENCESoftware Engineer
    -Microsoft - Manchester, UK.
    -December 2015 to Present
    -1. Microsoft Rewards Live dashboards:
    -Description:  -  Microsoft  rewards  is  loyalty  program  that  rewards  Users  for
    -browsing  and  shopping  online.  Microsoft  Rewards  members  can  earn  points  when
    -searching  with  Bing,  browsing  with  Microsoft  Edge  and  making  purchases  at  the
    -